Playing around with mistral and flag embeddings

In [3]:
import pandas as pd 
#pip install -U FlagEmbedding
#pip install langchain
#pip install mistral_inference
#pip install faiss-cpu

In [4]:
from FlagEmbedding import FlagAutoModel

embed_model = FlagAutoModel.from_finetuned('BAAI/bge-base-en-v1.5',
                                      query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                                      use_fp16=True)

In [5]:
# Removes extraneous columns and creates embeddings in the array

"""
df = pd.read_csv('Data/Medicine_Details.csv')
df.drop(columns={"Image URL", "Excellent Review %", 
                 "Average Review %", "Poor Review %"},inplace=True)
df['Combined_Info'] = 'Name: ' + df['Medicine Name'] + ' | Uses: ' + df['Uses'] + ' | Side Effects: ' + df['Side_effects']
embeddings = embed_model.encode(df['Combined_Info'].to_list())
df['embeddings'] = embeddings
df.head()
df.to_csv('test.csv', index=False)
"""

'\ndf = pd.read_csv(\'Data/Medicine_Details.csv\')\ndf.drop(columns={"Image URL", "Excellent Review %", \n                 "Average Review %", "Poor Review %"},inplace=True)\ndf[\'Combined_Info\'] = \'Name: \' + df[\'Medicine Name\'] + \' | Uses: \' + df[\'Uses\'] + \' | Side Effects: \' + df[\'Side_effects\']\nembeddings = embed_model.encode(df[\'Combined_Info\'].to_list())\ndf[\'embeddings\'] = embeddings\ndf.head()\ndf.to_csv(\'test.csv\', index=False)\n'

In [6]:
df = pd.read_csv('Data/test.csv')

In [ ]:
#Testing the embeddings with cosine_similarity

"""
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

query = "Tell me what Avastin's uses and side effects are"
query_embedded = model.encode([query])

similarity_scores = cosine_similarity(query_embedded, embeddings)
most_similar_idx = np.argmax(similarity_scores)
most_similar_row = df.iloc[most_similar_idx]
print("Most Relevant Row:")
print(most_similar_row)
"""

In [7]:
import faiss
import numpy as np

# Create and store faiss index

"""
embedding_array = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embedding_array.shape[1])
index.add(embedding_array)
faiss.write_index(index, "faiss_index.bin")
"""
index = faiss.read_index('faiss_index.bin')

In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.3"
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [9]:
query = "Tell me what Avastin's uses and side effects are"
query_embedding = embed_model.encode([query])
query_embedding_np = np.array(query_embedding).astype('float32')

D, I = index.search(query_embedding_np, 3)
retrieved_rows = df.iloc[I[0]]
retrieved_rows

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,Medicine Name,Composition,Uses,Side_effects,Manufacturer,Combined_Info,embeddings
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,Roche Products India Pvt Ltd,Name: Avastin 400mg Injection | Uses: Cancer ...,"[-0.029142677783966064, 0.04442485049366951, -..."
589,Avapar Tablet,Camylofin (50mg) + Diclofenac (50mg),Abdominal cramp,Nausea Vomiting Heartburn Stomach pain Indiges...,Khandelwal Laboratories Pvt Ltd,Name: Avapar Tablet | Uses: Abdominal cramp |...,"[0.00592622859403491, 0.03648262098431587, -0...."
412,Avas 40 Tablet,Atorvastatin (40mg),High cholesterolPrevention of Heart attack,Dyspepsia Abdominal pain Indigestion Diarrhea ...,Micro Labs Ltd,Name: Avas 40 Tablet | Uses: High cholesterol...,"[-0.028077833354473114, 0.04028759524226189, -..."


In [10]:
context = " ".join(retrieved_rows['Combined_Info'].tolist())
prompt = f"Based on the following context, answer the question:\n\nContext: {context}\n\nQuestion: {query}"
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=400)

In [ ]:
with torch.no_grad():
    output = model.generate(**inputs, max_new_tokens=100, do_sample=False)

In [ ]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
generated_text  

"Based on the following context, answer the question:\n\nContext: Name: Avastin 400mg Injection | Uses:  Cancer of colon and rectum Non-small cell lung cancer Kidney cancer Brain tumor Ovarian cancer Cervical cancer | Side Effects: Rectal bleeding Taste change Headache Nosebleeds Back pain Dry skin High blood pressure Protein in urine Inflammation of the nose Name: Avapar Tablet | Uses:  Abdominal cramp | Side Effects: Nausea Vomiting Heartburn Stomach pain Indigestion Loss of appetite Diarrhea Name: Avas 40 Tablet | Uses:  High cholesterolPrevention of Heart attack | Side Effects: Dyspepsia Abdominal pain Indigestion Diarrhea Joint pain Nasopharyngitis inflammation of the throat and nasal passages Nausea Pain in extremities Urinary tract infection Abnormal liver function tests\n\nQuestion: Tell me what Avastin's uses and side effects are.\n\nAnswer: Avastin's uses include cancer of colon and rectum, non-small cell lung cancer, kidney cancer, brain tumor, ovarian cancer, and cervical c

The return text is good, but need to keep in mind the data is very simple compared to promised MVP and model took 2 hours to generate the text.

Test 2

In [12]:
from huggingface_hub import snapshot_download
from pathlib import Path

mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

'C:\\Users\\Jason\\mistral_models\\7B-Instruct-v0.3'

In [13]:
# Might need to install certain things based on pc build and GPU
# pip install pip install torch==2.6.0+cu124 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

from mistral_inference.transformer import Transformer
from mistral_inference.generate import generate

from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest

In [14]:
import faiss
import numpy as np
import pandas as pd
import torch

In [16]:
torch.cuda.empty_cache()
from torch.utils.data import DataLoader

In [ ]:
# Not enough GPU storage to run this

with torch.no_grad():
    tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
    model = Transformer.from_folder(mistral_models_path, map_location='cpu')

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Jason\\mistral_models\\7B-Instruct-v0.3'

In [ ]:
df = pd.read_csv('Data/test.csv')

In [ ]:
from FlagEmbedding import FlagAutoModel

embed_model = FlagAutoModel.from_finetuned('BAAI/bge-base-en-v1.5',
                                      query_instruction_for_retrieval="Represent this sentence for searching relevant passages:",
                                      use_fp16=True)

In [6]:
index = faiss.read_index('faiss_index.bin')
query = "Tell me what Avastin's uses and side effects are"
query_embedding = embed_model.encode([query])
query_embedding_np = np.array(query_embedding).astype('float32')

D, I = index.search(query_embedding_np, 3)
retrieved_rows = df.iloc[I[0]]
retrieved_rows

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


,Medicine Name,Composition,Uses,Side_effects,Manufacturer,Combined_Info,embeddings
0,Avastin 400mg Injection,Bevacizumab (400mg),Cancer of colon and rectum Non-small cell lun...,Rectal bleeding Taste change Headache Noseblee...,Roche Products India Pvt Ltd,Name: Avastin 400mg Injection | Uses: Cancer ...,"[-0.029142677783966064, 0.04442485049366951, -..."
589,Avapar Tablet,Camylofin (50mg) + Diclofenac (50mg),Abdominal cramp,Nausea Vomiting Heartburn Stomach pain Indiges...,Khandelwal Laboratories Pvt Ltd,Name: Avapar Tablet | Uses: Abdominal cramp |...,"[0.00592622859403491, 0.03648262098431587, -0...."
412,Avas 40 Tablet,Atorvastatin (40mg),High cholesterolPrevention of Heart attack,Dyspepsia Abdominal pain Indigestion Diarrhea ...,Micro Labs Ltd,Name: Avas 40 Tablet | Uses: High cholesterol...,"[-0.028077833354473114, 0.04028759524226189, -..."


In [11]:
context = " ".join(retrieved_rows['Combined_Info'].tolist())
prompt = f"Based on the following context, answer the question:\n\nContext: {context}\n\nQuestion: {query}"

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=400)